In [ ]:
%matplotlib inline

import csv
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.stats
import seaborn as sns
import sklearn.preprocessing
import tensorflow as tf
import tick.hawkes
import pandas as pd

import vrnnframe as vrnnframe
from collections import defaultdict

In [ ]:
data_dir = '/y/data/processed_ddos'
trace_dirs = [
    'WSUa',
    'WSUb',
    'WSUc',
    'WSUd'
]
output_dir = '/y/adravi/ddos_chargen_2016_processed'
output_file = '/y/adravi/ddos_chargen_2016.npz'

if os.path.isfile(output_file):
    data = np.load(output_file)
    timestamps = data['timestamps']
    events = data['events']
else:
    print("No file found")

In [ ]:
dt_max = 0.01
t_train, dt_train, is_event_train, event_train = vrnnframe.preprocess(
    timestamps, [np.ones([timestamps_.shape[0], 1]) for timestamps_ in timestamps], dt_max
)

In [ ]:
n_latent = 10
n_hidden = 100
n_h = 100

lstm_cell = tf.keras.layers.LSTMCell(n_h)
x_network = tf.keras.Sequential([
    tf.keras.layers.Dense(n_hidden),
    tf.keras.layers.Activation('relu')
])
encoder_network = tf.keras.Sequential([
    tf.keras.layers.Dense(n_hidden),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(n_hidden),
    tf.keras.layers.Activation('relu')
])
prior_network = tf.keras.Sequential([
    tf.keras.layers.Dense(n_hidden), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(n_hidden),
    tf.keras.layers.Activation('relu')
])
z_network = tf.keras.Sequential([
    tf.keras.layers.Dense(n_hidden),
    tf.keras.layers.Activation('relu')
])
decoder_network = tf.keras.Sequential([
    tf.keras.layers.Dense(n_hidden),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(n_hidden),
    tf.keras.layers.Activation('relu')
])
recurrence_network = tf.keras.Sequential([
    tf.keras.layers.Dense(n_hidden),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(n_hidden),
    tf.keras.layers.Activation('relu')
])

vrnn = vrnnframe.VRNN(
    n_latent, dt_max, lstm_cell,
    x_network=x_network, encoder_network=encoder_network, decoder_network=decoder_network,
    prior_network=prior_network, z_network=z_network, recurrence_network=recurrence_network
)

vrnn = vrnnframe.VRNNModel(vrnn)
vrnn_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4)
vrnn.train(
    t_train, dt_train, is_event_train, event_train,
    lambda_=10, n_steps=10,
    batch_duration=100, batch_size=100, epochs=100,
    optimizer=vrnn_optimizer
)

In [ ]:
vrnn.train(
    t_train, dt_train, is_event_train, event_train,
    lambda_=10, n_steps=100,
    batch_duration=100, batch_size=100, epochs=10,
    optimizer=vrnn_optimizer
)

In [ ]:
intensity_pred, event_pred = vrnn.infer(
    t_train, dt_train, is_event_train, event_train,
    batch_size=200, return_intensity=True, return_event_prob=True
)

In [ ]:
dates = ['March 11', 'March 12', 'March 13', 'March 14', 'March 15', 'March 16', 'March 17']

fig, axs = plt.subplots(4, 2, figsize=(15, 10), constrained_layout=True)
axs = axs.flatten()
for i, date in enumerate(dates):
    axs[i].plot(t_train[i], intensity_pred[i], 'b')
    axs[i].set_title(date)

axs[-1].axis('off')

In [ ]:
encoder_mean, encoder_std, prior_mean, prior_std = vrnn.infer(
    t_test, dt_test, is_event_test, event_test, batch_size=500,
    return_encoder_mean=True, return_encoder_std=True, return_prior_mean=True, return_prior_std=True
)